In [1]:
import os, subprocess
import numpy as np
from profile_surface import get_neighbours, expand_to_fill, indices2surfaces
import tempfile
tmpdir=tempfile.gettempdir()




In [18]:

def load_coords(profile_file, n_lines=50000):
    """load streamline coords"""
    coords=[]
    with open(profile_file,'r') as profiles:
        for R in range(n_lines):
        #Read in each profile as 200 rows.
            for l in range(200):
                Line=profiles.readline().rstrip()
                LineSplit=Line.split(' ')
                coords.append(LineSplit[0:3])
            empty=profiles.readline()
    return np.array(coords).astype(float)


In [73]:
bigbrain_folder='/data1/users/kwagstyl/profile_areas/braincollection_profiles/bigbrain/'

def rescale_to_256(array):
    """ensure all values lie in range 0 - 255"""
    minimum=np.min(array[array!=0])
    array=(255*(array-minimum)/(np.max(array)-minimum)).astype(int)
    return array


def create_volume_block(coords,outputname=os.path.join(tmpdir,'volumeblock.mnc'),
                        examplemnc=os.path.join(bigbrain_folder,'full8_1000um.mnc'),
                        resolution=0.08, border = 1):
    """create mnc volume block of a specific resolution, limited by the coordinates
    coords : streamline coordinates to limit the block
    examplemnc: need a basic volume file to change
    voxel resolution: in microns
    border size: optional extra border to pad volume mm"""
    #create bounding tag file
    create_bounding_tag(coords)
    print('autocrop -resample -clob -from '+os.path.join(tmpdir,'bounding_tmp.tag') + ' -isoexpand ' + str(border)+
                    ' -isostep '+str(resolution)+' '+ examplemnc+' '+ outputname)
    subprocess.call('autocrop -resample -clob -from '+os.path.join(tmpdir,'bounding_tmp.tag') + ' -isoexpand ' + str(border)+
                    ' -isostep '+str(resolution)+' '+ examplemnc+' '+ outputname, shell=True)



def create_tag(coords,value=1,output=os.path.join(tmpdir,'tmp.tag')):
    """create tag file with values set as structure ids for tag to minc then expansion"""
    with open(output, 'w') as O:
        O.write("MNI Tag Point File\n")
        O.write("Volumes=1;\n")
        O.write("%Volume: volume.mnc\n")
        O.write("\n")
        O.write("Points =\n")
        for coordinate in coords:
                O.write('\n ' + ' '.join(map(str, coordinate)) + ' "'+str(value)+'"')
        O.write(';\n')

def create_bounding_tag(coords,output=os.path.join(tmpdir,'bounding_tmp.tag')):
    """create tag file with 8 coordinate to create the bounding box"""
    with open(output, 'w') as O:
        O.write("MNI Tag Point File\n")
        O.write("Volumes=1;\n")
        O.write("%Volume: volume.mnc\n")
        O.write("\n")
        O.write("Points =")
        xc,yc,zc=np.vstack((np.min(coords,axis=0),
        np.max(coords,axis=0))).T
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    coordinate=[xc[x],yc[y],zc[z]]
                    O.write('\n'+' '.join(map(str, coordinate)) + ' ""')
        O.write(';\n')

        
def tag_to_mnc(output_volumename,input_volume=os.path.join(tmpdir,'volumeblock.mnc'),tagfile=os.path.join(tmpdir,'tmp.tag'), max_iterations=1):
    """script that does the expansions from tag profiles"""
    subprocess.call('tagtominc '+input_volume+ ' '+ tagfile +' '+ os.path.join(tmpdir,'input.mnc'),shell=True)
    subprocess.call('dilate_volume '+ os.path.join(tmpdir,'input.mnc ') + output_volumename + ' 1 26 '+str(max_iterations),shell=True)
    return


In [81]:
def create_streamline_volume( streamline_coords, output_volumename,input_volume, hemi='left',max_iterations=1):
    """run set of scripts to generate dense saliency map"""
    print("writing values to tag file")
    create_tag(streamline_coords, value=1)
 #   print("creating a block volume to spec")
#    create_volume_block(coords,resolution=resolution)
  #  print("creating a cortical mask")
   # create_cortex_mask(hemi=hemi)
    create_volume_block(streamline_coords, examplemnc=input_volume,resolution=0.2)
    print("filling the volume")
    tag_to_mnc(output_volumename, max_iterations=max_iterations)



In [84]:
hemi='left'
coords=load_coords('/data1/users/kwagstyl/bigbrain/NeuralNetworks/BigBrainData/full8_1000um.mnc_'+hemi+'_profiles_raw.txt',n_lines=30000)

In [85]:
create_streamline_volume( coords, '/data1/users/kwagstyl/bigbrain/NeuralNetworks/surfdir/streamline_volume.mnc',
                        '/data1/users/kwagstyl/bigbrain/volumes/full8_100um.mnc',hemi=hemi, max_iterations=1)

writing values to tag file
autocrop -resample -clob -from /tmp/bounding_tmp.tag -isoexpand 1 -isostep 0.2 /data1/users/kwagstyl/bigbrain/volumes/full8_100um.mnc /tmp/volumeblock.mnc
filling the volume
tagtominc /tmp/volumeblock.mnc /tmp/tmp.tag /tmp/input.mnc
